<a href="https://colab.research.google.com/github/NaniDaChaman/Sparsh-coding-challenge-FA24/blob/main/Model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

## Git Config and startup

In [2]:
!git clone https://github.com/NaniDaChaman/Sparsh-coding-challenge-FA24.git

Cloning into 'Sparsh-coding-challenge-FA24'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 215 (delta 16), reused 1 (delta 0), pack-reused 182 (from 1)
Receiving objects: 100% (215/215), 7.74 MiB | 5.52 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [10]:
cd Sparsh-coding-challenge-FA24

/content/Sparsh-coding-challenge-FA24


In [11]:
ls

 Level_1/   Level_3/    Submissions/
 Level_2/   README.md  'Submissions\Political_Dataset_cleaning.ipynb'


In [12]:

!git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 368.89 KiB | 1.01 MiB/s, done.
From https://github.com/NaniDaChaman/Sparsh-coding-challenge-FA24
   d1be21b..5ff5630  main       -> origin/main
Updating d1be21b..5ff5630
Fast-forward
 Submissions/5nn_model | Bin 0 -> 2894193 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Submissions/5nn_model


In [3]:
!git config --global user.name "NaniDaChaman"
!git config --global user.email "nani00sp@gmail.com"

In [15]:
clf=None
encoder_decoder=None
donations_processed = pd.read_csv('/content/Sparsh-coding-challenge-FA24/Submissions/donations_processed.csv')
with open('/content/Sparsh-coding-challenge-FA24/Submissions/5nn_model', 'rb') as handle:
    clf = pickle.load(handle)
with open('/content/Sparsh-coding-challenge-FA24/Submissions/encoder_decoder.pickle', 'rb') as handle:
    encoder_decoder = pickle.load(handle)


In [16]:
encoder_decoder

{'committee_name': LabelEncoder(),
 'contributor_employer': LabelEncoder(),
 'contributor_occupation': LabelEncoder(),
 'election_type': LabelEncoder(),
 'fec_election_type_desc': LabelEncoder()}

In [17]:
clf

Pipeline(steps=[('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])

In [18]:
donations_processed.head(10)

,Unnamed: 0,committee_name,contributor_zip,contributor_employer,contributor_occupation,election_type,fec_election_type_desc,report_year,contribution_receipt_amount
0,1,12,37027.0,15,340,12,4,2023.0,416.67
1,2,31,27104.0,15,806,12,4,2023.0,84.00
2,3,31,27104.0,15,806,12,4,2023.0,84.00
3,4,31,27104.0,15,806,12,4,2023.0,84.00
4,5,2,37221.0,15,776,9,3,2023.0,3.00
5,6,208,37209.0,1,612,9,3,2020.0,16.00
6,7,297,37204.0,15,695,12,4,2023.0,20.22
7,8,12,37027.0,15,340,12,4,2023.0,416.67
8,9,80,37221.0,15,609,12,4,2023.0,50.00
9,10,297,37204.0,15,695,12,4,2023.0,20.22


## Showing Distribution of our classes

In [28]:
class_count=donations_processed[['committee_name','contributor_zip']].groupby('committee_name').count()

In [30]:
class_count.index=encoder_decoder['committee_name'].inverse_transform(class_count.index)

In [32]:
class_count

,contributor_zip
314 ACTION FUND,13
AB PAC,2
ACTBLUE,30035
ADAM FOR COLORADO,3
ADAM HOLLIER FOR CONGRESS,2
...,...
WILLIAM HASTON FOR CONGRESS,2
WIN THE ERA PAC,53
WINRED,1404
WOMEN WIN WISCONSIN PAC,1


**Since some of our classes have most of our data we employ Precision and Recall as metrics to measure our model**

## Recall

**TP/TP+FN** it gives our models ability to predict c correctly vs the time it should've predicted c correctly

In [39]:
#train test split
y = donations_processed['committee_name'].to_numpy()
X = donations_processed.iloc[0:,2:]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4, random_state=0)

In [43]:
y_pred=clf.predict(X_test.to_numpy())

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [48]:
recall_score(y_test, y_pred, average='macro')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.19355834409563472

In [50]:
recall_score(y_test, y_pred, average='micro')

0.9158586825350589

In [51]:
recall_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.9158586825350589

**Indicates that our model is reporting a lot of false negatives and maybe something we need to look into**

## Precision

**tp / (tp + fp)** our models ability to predict c correctly vs the times it predicted c incorrectly

In [52]:
precision_score(y_test, y_pred, average='macro')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.22074858540477657

In [53]:
precision_score(y_test, y_pred, average='micro')

0.9158586825350589

In [54]:
precision_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.9008990271180896

## Acknowledgement
 Precision and Recall : https://scikit-learn.org/dev/api/sklearn.metrics.html#module-sklearn.metrics

 Modeling : https://www.geeksforgeeks.org/k-nearest-neighbors-knn-regression-with-scikit-learn/


 PCA : https://www.geeksforgeeks.org/principal-component-analysis-pca/

 Model cleaning : https://towardsdatascience.com/so-youve-got-a-dataset-here-s-how-you-clean-it-5d0b04a2ed86

## Conclusion
**Our model seems to be overfitted to predict the classes which occur the most vs classes which appear the least**
